In [ ]:
# !pip install sentence-transformers pandas scikit-learn torch
# !pip install -U sentence_transformers
# !pip install huggingface_hub
# !pip install chromadb
# !pip install -qU cohere langchain_cohere

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

# 데이터셋 로드 및 분할 (Train,Val,Test)

In [ ]:
# 최적화 전
# 초기 학습 모델

import json
import torch
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation

# 1. 모델 및 데이터 로드 (경로는 여욱님 환경에 맞춰 확인해 주세요)
model_id = "google/embeddinggemma-300m"
model = SentenceTransformer(model_id)

def load_json_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

# 데이터 경로 다시 한 번 체크!
train_data = load_json_data('./data/train_dataset.json')
val_data = load_json_data('./data/val_dataset.json')

# 2. 학습용 데이터 객체(InputExample) 생성
train_examples = [
    InputExample(texts=[item['anchor'], item['positive'], item['negative']])
    for item in train_data
]

# 3. 데이터 로더 및 손실 함수 설정
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.MultipleNegativesRankingLoss(model)

# 4. 검증기(Evaluator) 설정
# 학습 중간에 모델의 실력이 얼마나 늘었는지 체크하는 도구입니다.
queries = {str(i): item['anchor'] for i, item in enumerate(val_data)}
corpus = {str(i): item['positive'] for i, item in enumerate(val_data)}
relevant_docs = {str(i): {str(i)} for i, item in enumerate(val_data)}

evaluator = evaluation.InformationRetrievalEvaluator(
    queries, corpus, relevant_docs, name="card-val-task"
)

# 5. 학습 실행 (model.fit 방식 - 모든 버전에서 호환됨)
print("🚀 [안전 모드] Gemma-300M 학습을 시작합니다...")

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=3,                  # 전체 데이터를 3번 훑습니다.
    warmup_steps=100,          # 초기 안정화 단계
    output_path="./models/gemma-300m-finetuned-card",
    evaluation_steps=200,      # 200번의 걸음마다 시험을 봅니다.
    save_best_model=True,
    show_progress_bar=True
)

print("✨ 모든 과정이 끝났습니다! ./models/gemma-300m-finetuned-card 폴더를 확인하세요.")

## Gemma-300m 기반 카드 혜택 검색 모델 파인튜닝

- 초경량 효율성: 파라미터 수가 약 3억 개로 적어, 매우 빠른 학습과 추론 속도를 보장합니다.
- 강력한 기본 성능: 작은 체급임에도 불구하고 Google의 최신 아키텍처가 적용되어, 문장의 맥락을 파악하는 능력이 뛰어납니다.
- RAG 최적화: 지시어를 이해하는 능력이 탁월하여, 검색 기반 생성 시스템의 핵심인 '문서 임베딩' 작업에 적합합니다.

Gemma 임베딩 모델은 특정 지시어(Instruction)가 포함될 때 최상의 성능을 발휘합니다. 이를 위해 쿼리와 문서에 각각 접두사를 붙여주는 전처리 함수를 사용합니다.
- Query: task: search result | query: {질문}
- Document: title: {카드명} | text: {혜택 내용}

In [ ]:
import json
import torch
import os
import gc
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation
from tqdm.autonotebook import tqdm

# 1. GPU 메모리 초기화 
gc.collect()
torch.cuda.empty_cache()

# 2. 모델 로드 및 그래디언트 체크포인팅 활성화

MODEL_ID = "google/embeddinggemma-300m"
OUTPUT_PATH = "./models/gemma-300m-4080super-extreme"
model = SentenceTransformer(MODEL_ID)
model[0].auto_model.gradient_checkpointing_enable()

# 3. Gemma 공식 지시어 포맷 함수
def format_query(text):
    return f"task: search result | query: {text}"

def format_doc(content, title="none"):
    return f"title: {title} | text: {content}"

def load_json_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

# 데이터 로드
train_data = load_json_data('./data/train_dataset.json')
val_data = load_json_data('./data/val_dataset.json')

# 4. 학습 데이터셋 구성 (Prefix 적용)
train_examples = []
for item in train_data:
    card_title = item.get('metadata', {}).get('card_name', '카드혜택')
    train_examples.append(InputExample(texts=[
        format_query(item['anchor']), 
        format_doc(item['positive'], title=card_title),
        format_doc(item['negative'], title=card_title)
    ]))


BATCH_SIZE = 128 
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=BATCH_SIZE)
train_loss = losses.MultipleNegativesRankingLoss(model)

# 6. 검증기 설정
val_queries = {str(i): format_query(item['anchor']) for i, item in enumerate(val_data)}
val_corpus = {str(i): format_doc(item['positive']) for i, item in enumerate(val_data)}
val_relevant_docs = {str(i): {str(i)} for i, item in enumerate(val_data)}

evaluator = evaluation.InformationRetrievalEvaluator(
    val_queries, val_corpus, val_relevant_docs, name="card-val-task"
)

# 7. 진행 상황 실시간 모니터링 콜백
def training_callback(score, epoch, steps):
    print(f"\n📊 [REPORT] Step: {steps} | Epoch: {epoch} | Accuracy@1: {score:.4f}")

# 8. 최적화 학습 실행
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=10,
    warmup_steps=100,
    optimizer_params={'lr': 2e-5},
    output_path=OUTPUT_PATH,
    evaluation_steps=20,     # 배치가 크므로 20스텝마다 점수 확인 (진행 상황 체크)
    save_best_model=True,
    show_progress_bar=True,
    use_amp=True,            # 혼합 정밀도 활성화
    callback=training_callback # 실시간 점수 출력
)

🔥 [Extreme Mode] RTX 4080 Super 가동! (Batch: 128)
🛠️ 활성화 기술: AMP(혼합정밀도) + Gradient Checkpointing


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss,Card-val-task Cosine Accuracy@1,Card-val-task Cosine Accuracy@3,Card-val-task Cosine Accuracy@5,Card-val-task Cosine Accuracy@10,Card-val-task Cosine Precision@1,Card-val-task Cosine Precision@3,Card-val-task Cosine Precision@5,Card-val-task Cosine Precision@10,Card-val-task Cosine Recall@1,Card-val-task Cosine Recall@3,Card-val-task Cosine Recall@5,Card-val-task Cosine Recall@10,Card-val-task Cosine Ndcg@10,Card-val-task Cosine Mrr@10,Card-val-task Cosine Map@100
20,No log,No log,0.603871,0.767742,0.832258,0.913548,0.603871,0.255914,0.166452,0.091355,0.603871,0.767742,0.832258,0.913548,0.752219,0.701285,0.705925
40,No log,No log,0.680000,0.829677,0.896774,0.958710,0.680000,0.276559,0.179355,0.095871,0.680000,0.829677,0.896774,0.958710,0.815290,0.769704,0.772001
49,No log,No log,0.669677,0.833548,0.899355,0.958710,0.669677,0.277849,0.179871,0.095871,0.669677,0.833548,0.899355,0.958710,0.811697,0.764829,0.767191
60,No log,No log,0.668387,0.832258,0.892903,0.962581,0.668387,0.277419,0.178581,0.096258,0.668387,0.832258,0.892903,0.962581,0.813547,0.766013,0.768089
80,No log,No log,0.677419,0.843871,0.883871,0.958710,0.677419,0.281290,0.176774,0.095871,0.677419,0.843871,0.883871,0.958710,0.817749,0.772779,0.775222
98,No log,No log,0.676129,0.849032,0.907097,0.961290,0.676129,0.283011,0.181419,0.096129,0.676129,0.849032,0.907097,0.961290,0.818403,0.772653,0.775015
100,No log,No log,0.681290,0.850323,0.910968,0.966452,0.681290,0.283441,0.182194,0.096645,0.681290,0.850323,0.910968,0.966452,0.823616,0.777909,0.779802
120,No log,No log,0.667097,0.843871,0.904516,0.966452,0.667097,0.281290,0.180903,0.096645,0.667097,0.843871,0.904516,0.966452,0.817409,0.769590,0.771451
140,No log,No log,0.676129,0.840000,0.899355,0.962581,0.676129,0.280000,0.179871,0.096258,0.676129,0.840000,0.899355,0.962581,0.817619,0.771388,0.773507
147,No log,No log,0.669677,0.847742,0.899355,0.960000,0.669677,0.282581,0.179871,0.096000,0.669677,0.847742,0.899355,0.960000,0.817036,0.771040,0.773590



📊 [REPORT] Step: 20 | Epoch: 0.40816326530612246 | Accuracy@1: 0.7522

📊 [REPORT] Step: 40 | Epoch: 0.8163265306122449 | Accuracy@1: 0.8153

📊 [REPORT] Step: 49 | Epoch: 1.0 | Accuracy@1: 0.8117

📊 [REPORT] Step: 60 | Epoch: 1.2244897959183674 | Accuracy@1: 0.8135

📊 [REPORT] Step: 80 | Epoch: 1.6326530612244898 | Accuracy@1: 0.8177

📊 [REPORT] Step: 98 | Epoch: 2.0 | Accuracy@1: 0.8184

📊 [REPORT] Step: 100 | Epoch: 2.0408163265306123 | Accuracy@1: 0.8236

📊 [REPORT] Step: 120 | Epoch: 2.4489795918367347 | Accuracy@1: 0.8174

📊 [REPORT] Step: 140 | Epoch: 2.857142857142857 | Accuracy@1: 0.8176

📊 [REPORT] Step: 147 | Epoch: 3.0 | Accuracy@1: 0.8170

📊 [REPORT] Step: 160 | Epoch: 3.2653061224489797 | Accuracy@1: 0.8183

📊 [REPORT] Step: 180 | Epoch: 3.673469387755102 | Accuracy@1: 0.8298

📊 [REPORT] Step: 196 | Epoch: 4.0 | Accuracy@1: 0.8313

📊 [REPORT] Step: 200 | Epoch: 4.081632653061225 | Accuracy@1: 0.8323

📊 [REPORT] Step: 220 | Epoch: 4.489795918367347 | Accuracy@1: 0.8306

📊 [

# 모델 성능 평가

InformationRetrievalEvaluator를 사용하여 검색 시스템의 핵심 구성 요소를 설정합니다.

- Queries: 사용자가 던질 수 있는 질문 예시.
- Corpus: 검색 대상이 되는 전체 카드 혜택 데이터베이스.
- Relevant Docs: 특정 질문에 대해 반드시 찾아내야 하는 '정답 문서' 매핑.

In [ ]:
import json
import os
from sentence_transformers import SentenceTransformer, evaluation

# 1. 모델 로드
FINAL_MODEL_PATH = "./models/gemma-300m-4080super-extreme"
final_model = SentenceTransformer(FINAL_MODEL_PATH)

# 2. 데이터 로드
def load_json_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

test_data = load_json_data('./data/test_dataset.json')

# 3. Evaluator 설정
test_queries = {str(i): item['anchor'] for i, item in enumerate(test_data)}
test_corpus = {str(i): item['positive'] for i, item in enumerate(test_data)}
test_relevant_docs = {str(i): {str(i)} for i, item in enumerate(test_data)}

# 평가 이름(name)을 기준으로 결과 딕셔너리의 키 값이 생성됩니다.
TASK_NAME = "card-test-task"
test_evaluator = evaluation.InformationRetrievalEvaluator(
    test_queries, test_corpus, test_relevant_docs, name=TASK_NAME
)

# 4. 최종 평가 실행
print("🔍 테스트 데이터를 이용한 심층 성능 분석을 시작합니다...")
results = test_evaluator(final_model)

# 5. 상세 지표 출력 로직
def print_detailed_report(res, name):
    
    metrics = [
        # [Accuracy@1] "한 방에 맞출 확률"
        # 사용자가 질문하자마자 리스트의 가장 첫 번째(1위)에 정답이 있을 확률입니다.
        ("Accuracy@1 (Hit Rate)", f"{name}_cosine_accuracy@1"),
        
        # [Accuracy@10] "10위권 안에 정답이 있을 확률"
        # 1위는 아니더라도 10위 안에만 정답이 있으면 성공으로 침. 시스템의 안정성을 보여줍니다.
        ("Accuracy@10 (Hit Rate)", f"{name}_cosine_accuracy@10"),
        
        # [MRR@10] "정답이 상단에 배치되는 능력"
        # 정답이 1등이면 1점, 2등이면 0.5점... 식으로 점수를 매겨 정답이 얼마나 앞쪽에 쏠려 있는지 측정합니다.
        ("MRR@10", f"{name}_cosine_mrr@10"),
        
        # [NDCG@10] "검색 결과의 종합적인 품질"
        # 정답 문서가 상단에 잘 정렬되어 있는지 점수를 매기는 지표로, 구글 같은 검색 엔진이 가장 중시합니다.
        ("NDCG@10", f"{name}_cosine_ndcg@10"),
        
        # [Precision@10] "추천된 10개 중 정답의 농도"
        # 뽑아준 10개 중에 진짜 정답이 몇 개 섞여 있는지 확인합니다. (정답이 1개면 0.1이 만점)
        ("Precision@10", f"{name}_cosine_precision@10"),
        
        # [Recall@10] "정답을 놓치지 않고 찾아낸 비율"
        # 전체 정답 중 모델이 10위 안에 건져 올린 비율입니다. 현재 Accuracy@10과 수치가 같습니다.
        ("Recall@10", f"{name}_cosine_recall@10"),
    ]
    
    print(f"\n{'='*25} [Gemma-300M 최종 검색 성적표] {'='*25}")
    for label, key in metrics:
        val = res.get(key, 0)
        print(f"✅ {label:<30} : {val:.4f}")
    print(f"{'='*78}\n")

print_detailed_report(results, TASK_NAME)

The tokenizer you are loading from './models/gemma-300m-4080super-extreme' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


🔍 테스트 데이터를 이용한 심층 성능 분석을 시작합니다...

========================= [Gemma-300M 최종 검색 성적표] =========================
✅ Accuracy@1 (Hit Rate)          : 0.7384
✅ Accuracy@10 (Hit Rate)         : 0.9613
✅ MRR@10                         : 0.8191
✅ NDCG@10                        : 0.8540
✅ Precision@10                   : 0.0961
✅ Recall@10                      : 0.9613



# Gemma-300M & ChromaDB 기반 카드 검색 시스템 구축

- 학습된 임베딩 모델을 사용하여 텍스트 데이터를 벡터화하고, 이를 ChromaDB에 저장하여 유사도 검색을 수행합니다.

- Vector DB: ChromaDB (데이터를 하드디스크에 영구 저장 및 관리)
- Embedding Model: 여욱님이 직접 튜닝한 Gemma-300M 모델 사용
- 핵심 설정: 벡터 간 거리 측정 방식을 코사인 유사도로 설정하여 문맥적 유사성 측정



In [2]:
import json
import os
import torch
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer

# 데이터 경로 및 모델 경로 설정
INPUT_FILE = './data/FINAL_MASTER_DATA_FIXED_7757.json'      # 7,757건의 카드 원본 데이터
MODEL_PATH = './models/gemma-300m-4080super-extreme'          # 학습시킨 Gemma 임베딩 모델
CHROMA_DB_PATH = './data/chroma_db'                         # 벡터 데이터가 저장될 폴더 경로


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"📦 모델을 로드 중입니다... (Device: {device})")

# SentenceTransformer를 통해 Gemma 모델을 메모리에 올립니다.
model = SentenceTransformer(MODEL_PATH, device=device)

# ==========================================
# ChromaDB 커스텀 임베딩 함수 정의
# ==========================================
# ChromaDB가 스스로 임베딩을 하지 않고, Gemma 모델을 사용하도록 다리를 놓아주는 클래스입니다.
class GemmaEmbeddingFunction(embedding_functions.EmbeddingFunction):
    def __call__(self, input_texts):
    
        passages = [f"title: none | text: {text}" for text in input_texts]
        embeddings = model.encode(passages, convert_to_tensor=False)
        return embeddings.tolist()  # DB 저장을 위해 리스트 형태로 반환

# ==========================================
# ChromaDB 초기화 및 데이터 인덱싱
# ==========================================
# 데이터를 하드디스크에 영구 저장하는 클라이언트를 생성합니다.
client = chromadb.PersistentClient(path=CHROMA_DB_PATH)
embedding_fn = GemmaEmbeddingFunction()

# 'card_benefits'라는 이름의 데이터 저장 공간(컬렉션)을 만듭니다.
# metadata={"hnsw:space": "cosine"}는 벡터 간의 거리를 '코사인 유사도'로 계산하겠다는 설정입니다.
collection = client.get_or_create_collection(
    name="card_benefits",
    embedding_function=embedding_fn,
    metadata={"hnsw:space": "cosine"} 
)

# DB에 데이터가 하나도 없다면 최초 1회 인덱싱을 수행합니다.
if collection.count() == 0:
    print("인덱싱을 시작합니다...")
    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        master_data = json.load(f)
    
    # 7,757건의 데이터를 DB 형식에 맞게 분리합니다.
    documents = [item['embedding_input'] for item in master_data]   # 검색 대상 텍스트
    metadatas = [item['metadata'] for item in master_data]          # 카드 이름, 회사 등 부가 정보
    
    # 나중에 검색 결과에서 바로 요약을 보여주기 위해 메타데이터에 미리 넣어둡니다.
    for i, m in enumerate(metadatas):
        m['summary'] = master_data[i]['ai_structured']['summary']
    
    # 각 데이터의 고유 ID 생성 (card_0, card_1...)
    ids = [f"card_{i}" for i in range(len(master_data))]

    #500개씩 나눠서 저장합니다.
    batch_size = 500
    for i in range(0, len(documents), batch_size):
        collection.add(
            documents=documents[i:i+batch_size],
            metadatas=metadatas[i:i+batch_size],
            ids=ids[i:i+batch_size]
        )
    print(f"✅ 총 {collection.count()}건의 데이터 저장 완료!")
else:
    print(f"🔄 기존 ChromaDB를 로드했습니다. (저장된 데이터: {collection.count()}건)")

# ==========================================
# 다중 의도 교차 검색 (AND 로직)
# ==========================================
def find_combined_card_chroma(query_list, top_k=10, search_depth=100):
    """
    query_list: ['의도1', '의도2'] 검색 키워드 리스트
    top_k: 최종적으로 몇 개를 보여줄 것인가
    search_depth: 각 의도별로 상위 몇 개까지 훑어볼 것인가 (깊을수록 정확함)
    """
    intent_hits = []

    # --- 1단계: 각 의도별 개별 검색 ---
    for sub_query in query_list:
        # 질문용 공식 프롬프트 적용 (학습 시 instruction과 일치시켜야 함)
        query_text = f"task: search result | query: {sub_query}"
        
        # DB에서 유사한 카드 search_depth만큼 추출
        results = collection.query(
            query_texts=[query_text],
            n_results=search_depth,
            include=['metadatas', 'distances']
        )
        
        current_hits = {}
        for i in range(len(results['ids'][0])):
            meta = results['metadatas'][0][i]
            name = meta['card_name']
            
            # [유사도 계산] 거리가 0이면 점수는 1점(만점), 거리가 1이면 점수는 0점
            score = 1 - results['distances'][0][i]
            
            # 같은 카드가 중복 검색될 경우 가장 점수가 높은 것만 기록
            if name not in current_hits:
                current_hits[name] = {
                    "score": score,
                    "benefit": meta.get('summary', '혜택 정보 없음'),
                    "corp": meta['corp']
                }
        intent_hits.append(current_hits)

    # --- 2단계: 교집합(Intersection) 찾기 ---
    # 모든 의도 리스트에 이름이 공통적으로 존재하는 카드만 골라냅니다.
    common_names = set(intent_hits[0].keys()) # 첫 번째 의도 결과들
    for hits in intent_hits[1:]:
        common_names &= set(hits.keys())      # 이후 의도들과 계속 겹치는 것만 남김

    # --- 3단계: 결과 병합 및 점수 합산 ---
    final_recommendations = []
    for name in common_names:
        # 각 의도에서 받은 유사도 점수를 모두 더합니다 (종합 점수)
        total_score = sum(hits[name]['score'] for hits in intent_hits)
        # 각 의도 검색 시 발견된 혜택 요약들을 중복 없이 수집합니다.
        all_benefits = list(set([hits[name]['benefit'] for hits in intent_hits]))
        corp = intent_hits[0][name]['corp']
        
        final_recommendations.append({
            "name": name,
            "corp": corp,
            "score": total_score,
            "benefits": all_benefits
        })

    # --- 4단계: 최종 정렬 및 출력 ---
    # 종합 점수(score)가 높은 순서대로 상위 top_k개 선정
    final_recommendations = sorted(final_recommendations, key=lambda x: x['score'], reverse=True)[:top_k]

    print(f"\n🔍 분석된 의도: {query_list}")
    print(f"✅ 교집합 필터링 결과: {len(common_names)}건 발견")
    print("="*85)
    
    for i, res in enumerate(final_recommendations):
        # 최종 결과 출력
        print(f"{i+1}. [{res['corp']}] {res['name']} (종합 점수: {res['score']:.4f})")
        for b in res['benefits']:
            print(f"     - {b}")
        print("-" * 85)

# ==========================================
# 5. 실행 테스트
# ==========================================
# 사용자의 복합적인 질문을 두 개의 핵심 의도로 나누어 입력합니다.
intents = [
    "배달의민족 요기요 쿠팡이츠 배달 음식 할인 혜택", 
    "넷플릭스 디즈니플러스 왓챠 OTT 스트리밍 할인 혜택"
]

# 함수 호출
find_combined_card_chroma(intents)

📦 모델을 로드 중입니다... (Device: cuda)


The tokenizer you are loading from './models/gemma-300m-4080super-extreme' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
C:\Users\Jeon\AppData\Local\Temp\ipykernel_38488\3610440556.py:36: DeprecationWarning: The class GemmaEmbeddingFunction does not implement __init__. This will be required in a future version.
  embedding_fn = GemmaEmbeddingFunction()


🔄 기존 ChromaDB를 로드했습니다. (저장된 데이터: 7757건)

🔍 분석된 의도: ['배달의민족 요기요 쿠팡이츠 배달 음식 할인 혜택', '넷플릭스 디즈니플러스 왓챠 OTT 스트리밍 할인 혜택']
✅ 교집합 필터링 결과: 28건 발견
1. [롯데카드] 디지로카 Monaco (종합 점수: 0.8458)
     - 넷플릭스, 유튜브프리미엄, 왓챠, 멜론, 지니뮤직에서 50% 결제일 할인이 제공됩니다.
     - 배달의 민족, 쿠팡이츠, 요기요에서 5% 결제일 할인 혜택 제공.
-------------------------------------------------------------------------------------
2. [현대카드] 현대카드Z ontact (종합 점수: 0.8417)
     - 배달 앱 이용금액의 10% 청구 할인 혜택 제공
     - 디지털 콘텐츠 서비스 이용금액의 30% 청구 할인 제공
-------------------------------------------------------------------------------------
3. [롯데카드] 뱅크샐러드 빨대카드 (종합 점수: 0.8276)
     - 넷플릭스, 유튜브, 왓챠, 멜론에서 50% 결제일 할인 혜택 제공.
     - 배달의민족과 요기요에서 1만원 이상 결제 시 1천원 할인 혜택이 제공됩니다.
-------------------------------------------------------------------------------------
4. [하나카드] 원더카드 2.0 DAILY (종합 점수: 0.7757)
     - 딜리버리 서비스에서 10% 할인을 제공하는 카드 혜택입니다.
     - 영상 스트리밍 서비스에 대해 최대 40% 할인 혜택을 제공합니다.
-------------------------------------------------------------------------------------
5. [KB국민카드] 레

In [ ]:
import json
import os
import torch
import chromadb
import cohere
from dotenv import load_dotenv
from openai import OpenAI
from sentence_transformers import SentenceTransformer
from chromadb.utils import embedding_functions

# 1. 환경 변수 로드 (API 키)
load_dotenv()
OPENAI_KEY = os.getenv("OPENAI_API_KEY")
COHERE_KEY = os.getenv("COHERE_API_KEY")

# 2. 로컬 경로 및 설정
INPUT_FILE = './data/FINAL_MASTER_DATA_FIXED_7757.json'
MODEL_PATH = './models/gemma-300m-4080super-extreme'    
CHROMA_DB_PATH = './data/chroma_db'

# 3. 모델 및 클라이언트 초기화
device = "cuda" if torch.cuda.is_available() else "cpu"
bi_encoder = SentenceTransformer(MODEL_PATH, device=device)
openai_client = OpenAI(api_key=OPENAI_KEY)
co = cohere.ClientV2(api_key=COHERE_KEY)

# 4. 데이터 리하이드레이션 맵 생성 (ID 기반 복원)
with open(INPUT_FILE, 'r', encoding='utf-8') as f:
    master_data = json.load(f)

card_master_map = {}
for item in master_data:
    c_id = item['metadata']['card_id']
    if c_id not in card_master_map:
        card_master_map[c_id] = {
            "name": item['metadata']['card_name'],
            "corp": item['metadata']['corp'],
            "full_details": [item['content']], 
            "structured": item['ai_structured']
        }
    else:
        card_master_map[c_id]["full_details"].append(item['content'])

# 5. ChromaDB 연결 및 임베딩 함수 설정
class GemmaEmbeddingFunction(embedding_functions.EmbeddingFunction):
    def __call__(self, input_texts):
        passages = [f"title: none | text: {text}" for text in input_texts]
        return bi_encoder.encode(passages, convert_to_tensor=False).tolist()

chroma_client = chromadb.PersistentClient(path=CHROMA_DB_PATH)
collection = chroma_client.get_collection(
    name="card_benefits", 
    embedding_function=GemmaEmbeddingFunction()
)

# ==========================================
# [HELPER] 동적 의도 추출 함수
# ==========================================
def extract_search_intents(user_query):
    """사용자 질문에서 검색용 핵심 키워드를 동적으로 추출합니다."""

    
    prompt = f"""사용자의 질문을 분석하여, 카드 혜택 데이터베이스 검색에 가장 적합한 2~3개의 상세 검색 문장을 생성해줘.
            [가이드]
            - 단순한 키워드가 아닌, 혜택의 종류와 대표 브랜드가 포함된 짧은 문장으로 만드세요.
            - 예: "커피 할인되고 교통비 아끼는 카드"
            - 결과: ["스타벅스 투썸 등 커피 전문점 할인 혜택", "버스 지하철 택시 대중교통 이용 할인 혜택"]
            
            질문: {user_query}
            형식: ["문장1", "문장2"] (JSON 리스트로만 응답하세요)"""
    
    response = openai_client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1,  # 검색 키워드 추출은 '정확도'가 생명이므로 낮은 값을 추천
        top_p=0.9         # 답변의 일관성을 높이기 위한 보조 설정
    )
    try:
        return json.loads(response.choices[0].message.content)
    except:
        return [user_query]

# ==========================================
# [CORE] 통합 RAG 함수
# ==========================================
def card_concierge_rag(user_query, top_n=3):
    
    # [Step 1] 동적 의도 추출
    search_intents = extract_search_intents(user_query)
    print(f"추출된 키워드: {search_intents}")

    # [Step 2] ChromaDB 교차 검색 (AND 로직)
    intent_hits = []
    for intent in search_intents:
        query_text = f"task: search result | query: {intent}"
        results = collection.query(
            query_texts=[query_text],
            n_results=200,
            include=['metadatas']
        )
        ids = {m['card_id'] for m in results['metadatas'][0]}
        intent_hits.append(ids)

    # 교집합 필터링
    common_ids = intent_hits[0]
    for h in intent_hits[1:]:
        common_ids &= h

    if not common_ids:
        print("⚠️ 교집합 결과가 없어 단일 의도 결과로 대체합니다.")
        common_ids = intent_hits[0]

    # [Step 3] Cohere v3.5 리랭킹
    candidate_ids = list(common_ids)
    rerank_docs = []
    for c_id in candidate_ids:
        card = card_master_map[c_id]
        context = f"카드명: {card['name']} | 상세혜택: {' '.join(card['full_details'])}"
        rerank_docs.append(context)

    print(f"Cohere v3.5 리랭킹 중... (후보: {len(rerank_docs)}개)")
    rerank_response = co.rerank(
        model="rerank-v3.5",
        query=user_query,
        documents=rerank_docs,
        top_n=top_n
    )

# [Step 4] 데이터 복원 및 LLM 답변 생성 (GPT-5-Nano)
    final_context = ""
    for i, hit in enumerate(rerank_response.results):
        best_id = candidate_ids[hit.index]
        card = card_master_map[best_id]
        # 순위 정보와 함께 컨텍스트 구성
        final_context += f"### [Top {i+1}] {card['name']} ({card['corp']})\n"
        final_context += f"- 실적조건: {card['structured']['condition']}\n"
        final_context += f"- 상세 데이터: {' / '.join(card['full_details'])}\n\n"

    prompt = f"""당신은 대한민국 최고의 '신용/체크카드 추천 전문가'입니다. 
    제공된 [카드 데이터]를 분석하여 사용자의 질문에 전문적이면서도 다정하게 답변하세요.

    사용자 질문: {user_query}

    [카드 데이터]
    {final_context}

    [답변 작성 가이드]
    1. **전문가 인사**: 사용자의 소비 니즈를 정확히 짚어주며, 전문가로서 신뢰감 있고 다정한 인사를 건네세요.
    2. **Top 3 목록**: 추천하는 상위 3개 카드의 이름과 카드사를 데이터 순서대로 명확히 나열하세요.
    3. **Top 1 심층 분석**: 1위로 선정된 카드가 사용자의 질문과 관련된 혜택을 구체적으로 어떻게 제공하는지(수치, 할인 한도, 실적 조건 등) 전문가의 시선에서 상세히 설명하세요.
    4. **실전 조언**: 1위 카드를 사용할 때 주의해야 할 전월 실적외 항목이나 혜택 제외 대상 등을 비서처럼 세심하게 조언해 주세요.
    5. **톤앤매너**: 표는 절대 사용하지 마세요. 읽기 편한 문장과 불렛 포인트를 사용하고, 사용자가 혜택을 충분히 이해할 수 있도록 전체 분량을 적절히 조절하세요."""

    # gpt-4.1-mini 호출 (파라미터 최적화)
    response = openai_client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {
                "role": "system", 
                "content": "당신은 제공된 데이터의 순위를 엄격히 준수하며, 정확한 수치에 기반해 조언하는 신용카드 추천 전문가 Gemma-Bot입니다."
            },
            {"role": "user", "content": prompt}
        ],
        temperature=0.1,  # 일관성 있는 답변 유도
        top_p=0.1,        # 순위 및 팩트 고정
        max_tokens=1000   # 적절한 분량 확보
    )
    return response.choices[0].message.content

# ==========================================
# 실행부
# ==========================================
if __name__ == "__main__":
    user_q = "사회초년생인데 배달의민족 요기요 쿠팡이츠 배달 음식 할인 혜택이 있고, 넷플릭스 디즈니플러스 왓챠 OTT 스트리밍 할인 혜택도 있는 카드를 추천해줘"
    
    result = card_concierge_rag(user_q)
    print(result)

The tokenizer you are loading from './models/gemma-300m-4080super-extreme' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
C:\Users\Jeon\AppData\Local\Temp\ipykernel_38488\916593743.py:53: DeprecationWarning: The class GemmaEmbeddingFunction does not implement __init__. This will be required in a future version.
  embedding_function=GemmaEmbeddingFunction()


추출된 키워드: ['배달의민족 요기요 쿠팡이츠 배달 음식 할인 혜택', '넷플릭스 디즈니플러스 왓챠 OTT 스트리밍 할인 혜택']
Cohere v3.5 리랭킹 중... (후보: 48개)
안녕하세요, 전여욱님! 사회초년생으로서 배달 음식과 OTT 스트리밍 할인 혜택을 원하신다니, 꼭 맞는 카드를 추천해드릴게요. 배달의민족, 요기요, 쿠팡이츠 같은 배달앱과 넷플릭스, 디즈니플러스, 왓챠 등 OTT 스트리밍 할인까지 챙길 수 있는 카드 Top 3를 소개해드리겠습니다.

1. 삼성 iD SELECT ALL 카드 (삼성카드)  
2. 원더카드 2.0 DAILY (하나카드)  
3. 레고랜드매니아카드 (KB국민카드)  

특히 1위인 삼성 iD SELECT ALL 카드는 배달앱과 OTT 스트리밍 할인 혜택이 아주 알차서 전여욱님께 강력 추천드려요.

- 배달앱(배달의민족, 요기요, 쿠팡이츠)에서 7% 할인 혜택을 드립니다.  
- 넷플릭스, 디즈니+, 유튜브, 티빙 등 디지털콘텐츠 정기결제 시 50% 할인도 제공해요. (월 최대 5,000원 한도)  
- 배달앱 할인과 OTT 할인 모두 전월 이용금액 40만원 이상 시 적용되며, 월 최대 할인한도는 배달앱 7,000원, OTT 5,000원입니다.  
- 배달앱 할인은 공식 앱이나 홈페이지 결제 건에 한하며, 가맹점 직접 결제는 제외됩니다.  
- 이외에도 아파트 관리비, 통신비, 교육비 등 다양한 생활 할인 옵션을 매월 선택해 혜택을 맞춤 설정할 수 있어요.  

실전 팁을 드리자면, 삼성 iD SELECT ALL 카드는 전월 실적 40만원 이상부터 혜택이 제공되니, 매달 카드 사용 금액을 체크해 주세요. 그리고 할인 제외 항목이 꽤 있으니, 공과금, 상품권 구매, 무이자 할부, 간편결제 일부 등은 할인 대상에서 제외된다는 점도 꼭 기억해 주세요. 특히 배달앱과 OTT 할인은 공식 앱이나 홈페이지를 통한 정기결제에만 적용되니, 결제 방법을 꼼꼼히 확인하시면 좋습니다.

전여욱님께서 배달과 OTT를 자주 이용하신다면 삼성 

In [ ]:
# 리랭킹 제외 버전
import json
import os
import torch
import chromadb
from dotenv import load_dotenv
from openai import OpenAI
from sentence_transformers import SentenceTransformer
from chromadb.utils import embedding_functions

# 1. 환경 변수 로드 (OpenAI API 키 등)
load_dotenv()
OPENAI_KEY = os.getenv("OPENAI_API_KEY")

# 2. 로컬 경로 및 설정
INPUT_FILE = './data/FINAL_MASTER_DATA_FIXED_7757.json'
MODEL_PATH = './models/gemma-300m-4080super-extreme'    # 학습시킨 Gemma 모델
CHROMA_DB_PATH = './data/chroma_db'

# 3. 모델 및 클라이언트 초기화
device = "cuda" if torch.cuda.is_available() else "cpu"
bi_encoder = SentenceTransformer(MODEL_PATH, device=device)
openai_client = OpenAI(api_key=OPENAI_KEY)

# 4. 데이터 리하이드레이션 맵 생성 (ID 기반 상세 데이터 복원용)
with open(INPUT_FILE, 'r', encoding='utf-8') as f:
    master_data = json.load(f)

card_master_map = {}
for item in master_data:
    c_id = item['metadata']['card_id']
    if c_id not in card_master_map:
        card_master_map[c_id] = {
            "name": item['metadata']['card_name'],
            "corp": item['metadata']['corp'],
            "full_details": [item['content']], 
            "structured": item['ai_structured']
        }
    else:
        card_master_map[c_id]["full_details"].append(item['content'])

# 5. ChromaDB 연결 및 커스텀 임베딩 함수 설정
class GemmaEmbeddingFunction(embedding_functions.EmbeddingFunction):
    def __call__(self, input_texts):
        # 학습 시 사용한 접두사(Prefix) 적용
        passages = [f"title: none | text: {text}" for text in input_texts]
        return bi_encoder.encode(passages, convert_to_tensor=False).tolist()

chroma_client = chromadb.PersistentClient(path=CHROMA_DB_PATH)
collection = chroma_client.get_collection(
    name="card_benefits", 
    embedding_function=GemmaEmbeddingFunction()
)

# ==========================================
# [HELPER] 동적 의도 추출 함수 (GPT-4.1 mini 사용)
# ==========================================
def extract_search_intents(user_query):
    """사용자의 복합 질문을 검색에 최적화된 개별 의도로 분리합니다."""
    prompt = f"""사용자의 질문을 분석하여, 카드 혜택 데이터베이스 검색에 가장 적합한 2~3개의 상세 검색 문장을 생성해줘.
            [가이드]
            - 단순 키워드가 아닌 브랜드를 포함한 구체적인 혜택 문장으로 만드세요.
            - 예: "스타벅스 커피 할인", "배민/요기요 배달 할인"
            
            질문: {user_query}
            형식: ["문장1", "문장2"] (JSON 리스트로만 응답하세요)"""
    
    response = openai_client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    try:
        return json.loads(response.choices[0].message.content)
    except:
        return [user_query]

# ==========================================
# [CORE] 통합 RAG 함수 (리랭킹 제외 최적화 버전)
# ==========================================
def card_concierge_rag(user_query, top_n=3):
    print(f"🔍 질문 분석 중...")
    search_intents = extract_search_intents(user_query)
    print(f"🎯 추출된 검색 의도: {search_intents}")

    # [Step 1] 각 의도별 검색 및 점수 합산
    card_scores = {} # {card_id: [score1, score2, ...]}

    for intent in search_intents:
        query_text = f"task: search result | query: {intent}"
        results = collection.query(
            query_texts=[query_text],
            n_results=1000,  # 후보군 수집
            include=['metadatas', 'distances']
        )
        
        for i in range(len(results['metadatas'][0])):
            m = results['metadatas'][0][i]
            dist = results['distances'][0][i]
            c_id = m['card_id']
            # 코사인 거리(Distance)를 유사도 점수(Score)로 변환 (1에 가까울수록 유사)
            score = 1 - dist
            
            if c_id not in card_scores:
                card_scores[c_id] = []
            card_scores[c_id].append(score)

    # [Step 2] 교집합(Intersection) 기반 최종 후보 선정
    final_candidates = []
    num_required_intents = len(search_intents)

    for c_id, scores in card_scores.items():
        # 모든 검색 의도에서 공통적으로 발견된 카드만 필터링 (AND 로직)
        if len(scores) == num_required_intents:
            avg_score = sum(scores) / num_required_intents
            final_candidates.append((c_id, avg_score))

    # 교집합 결과가 너무 적으면 단일 의도 상위 결과로 보완
    if len(final_candidates) < top_n:
        print("⚠️ 모든 조건을 만족하는 카드가 부족하여 유사도 기반으로 보완합니다.")
        for c_id, scores in card_scores.items():
            if (c_id, sum(scores)/len(scores)) not in final_candidates:
                final_candidates.append((c_id, max(scores)))

    # 점수 높은 순 정렬
    final_candidates = sorted(final_candidates, key=lambda x: x[1], reverse=True)[:top_n]

    # [Step 3] 데이터 복원 및 컨텍스트 구성
    final_context = ""
    for i, (c_id, score) in enumerate(final_candidates):
        card = card_master_map[c_id]
        final_context += f"### [Top {i+1}] {card['name']} ({card['corp']})\n"
        final_context += f"- 검색 매칭 점수: {score:.4f}\n"
        final_context += f"- 실적 조건: {card['structured']['condition']}\n"
        final_context += f"- 상세 혜택: {' / '.join(card['full_details'][:2])}\n\n" # 가독성을 위해 상위 2개 혜택 위주

    # [Step 4] 최종 답변 생성 (GPT-5-mini 사용)
    prompt = f"""당신은 전여욱님의 든든한 '금융 비서 Gemma-Bot'입니다. 
제공된 [카드 데이터]를 분석하여 사용자의 질문에 다정하고 명확하게 답변하세요.

사용자 질문: {user_query}

[카드 데이터]
{final_context}

[답변 작성 가이드]
1. **친절한 인사**: 사용자의 상황(사회초년생 등)을 공감하며 시작하세요.
2. **Top 3 목록**: 추천하는 카드의 이름과 카드사를 나열하세요.
3. **Top 1 집중 분석**: 1위 카드의 혜택 수치와 한도를 아주 상세히 설명하세요.
4. **실전 팁**: 전월 실적 제외 항목 등 주의사항을 조언하세요.
5. **톤앤매너**: 표 없이 불렛 포인트를 사용하고, 전문적이면서도 따뜻한 어조를 유지하세요."""

    response = openai_client.chat.completions.create(
        model="gpt-5-mini",
        messages=[
            {"role": "system", "content": "데이터 기반의 정확하고 다정한 금융 상담사입니다."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content

# ==========================================
# 실행부
# ==========================================
if __name__ == "__main__":
    user_q = "사회초년생인데 배달의민족 요기요 쿠팡이츠 배달 음식 할인 혜택이 있고, 넷플릭스 디즈니플러스 왓챠 OTT 스트리밍 할인 혜택도 있는 카드를 추천해줘"
    
    result = card_concierge_rag(user_q)
    print("\n✨ [금융 비서 Gemma-Bot 답변]:\n")
    print(result)

The tokenizer you are loading from './models/gemma-300m-4080super-extreme' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
C:\Users\Jeon\AppData\Local\Temp\ipykernel_38488\2004021981.py:52: DeprecationWarning: The class GemmaEmbeddingFunction does not implement __init__. This will be required in a future version.
  embedding_function=GemmaEmbeddingFunction()


🔍 질문 분석 중...
🎯 추출된 검색 의도: ['배달의민족 요기요 쿠팡이츠 배달 음식 할인', '넷플릭스 디즈니플러스 왓챠 OTT 스트리밍 할인']

✨ [금융 비서 Gemma-Bot 답변]:

안녕하세요 전여욱님 — 사회초년생이라 카드 고르기 고민 많으시죠. 차근차근 도와드릴게요. 먼저 제공해주신 카드 데이터 기준으로 분석한 결과와 실전 팁을 정리해 드립니다.

- Top 3 카드 (제공된 데이터 기준)
  - 럭키 유카드 (코나카드)
  - 아모레퍼시픽 IBK카드 (IBK기업은행)
  - I-Mileage (아시아나) (IBK기업은행)

- 중요한 확인사항
  - 제공된 카드 데이터에는 요청하신 ‘배달앱(배달의민족·요기요·쿠팡이츠) 할인’이나 ‘OTT(넷플릭스·디즈니플러스·왓챠) 스트리밍 할인’이 명시된 카드는 없습니다. 따라서 이 데이터 내에서는 배달 할인과 OTT 할인을 동시에 충족하는 카드를 추천드리기 어렵습니다.

- Top 1(럭키 유카드) 상세 혜택·한도
  - 발급사: 코나카드
  - 전월 실적: 전월 실적 조건 없음 (데이터상 별도 실적 필요 없음)
  - 혜택 항목 및 수치
    - 커피빈 15% 할인 — 기본 혜택으로, 데이터상 전월 실적 없이 적용되는 것으로 표기되어 있습니다.
    - 프로모션: 1만원 이상 결제할 때마다 ‘4개 행운번호 럭키권’ 1장 발급 — 결제 1만원 단위로 럭키권을 지급(프로모션 관련 안내는 유튜브 ‘럭키로코쇼’ 등에서 확인 가능)
  - 한도·횟수
    - 데이터에 별도의 월별/연간 할인한도나 횟수 제한은 명시되어 있지 않습니다. (따라서 실제 약관에서 한도·제외항목을 반드시 확인하세요.)
  - 기타
    - 데이터에선 커피 혜택과 프로모션만 명시되어 있으며, 배달·OTT 관련 혜택은 표기되어 있지 않습니다.

- 실전 팁 (꼭 확인하세요)
  - 제공 데이터상 전월 실적 없음으로 표기되어 있지만, 실제로는 특정 프로모션이나 가맹점·결제수단(앱결제·간편결제·정기결제 등)에 따라 적용 여부가 달라질 수

In [21]:
def print_all_card_details(target_card_id):
    # 1. 해당 ID에 맞는 모든 조각(fragments) 찾기
    card_fragments = [item for item in master_data if item['metadata']['card_id'] == target_card_id]
    
    if not card_fragments:
        print(f"❌ ID {target_card_id}에 해당하는 카드를 찾을 수 없습니다.")
        return

    # 2. 공통 정보 출력 (첫 번째 조각 기준)
    base_info = card_fragments[0]['metadata']
    print(f"\n{'='*30} CARD FACT CHECK {'='*30}")
    print(f"💳 카드명: {base_info['card_name']} ({base_info['corp']})")
    print(f"🆔 Card ID: {base_info['card_id']}")
    print(f"💰 연회비: {base_info['annual_fee']}")
    print(f"📉 최소실적: {base_info['min_performance']}원")
    print(f"{'='*77}\n")

    # 3. 각 혜택 카테고리별 상세 내용 출력
    for i, fragment in enumerate(card_fragments):
        meta = fragment['metadata']
        ai = fragment['ai_structured']
        
        print(f"[{i+1}] 카테고리: {meta['main_category']} > {meta['sub_category']} ({meta['category']})")
        print(f"   📝 AI 요약: {ai['summary']}")
        print(f"   🏢 주요 가맹점: {', '.join(ai['merchants'])}")
        print(f"   ✅ 상세 조건: {ai['condition']}")
        print(f"   💡 혜택 상세: {ai['benefit_detail']}")
        
        # 실적 제외 항목이나 유의사항이 있다면 추가 출력
        if ai.get('performance_exclusions') and ai['performance_exclusions'][0] != "정보 없음":
            print(f"   ⚠️ 실적 제외: {ai['performance_exclusions']}")
        
        print("-" * 77)

print_all_card_details(2885)


============================== CARD FACT CHECK ==============================
💳 카드명: 삼성 iD SELECT ALL 카드 (삼성카드)
🆔 Card ID: 2885
💰 연회비: 국내전용 [20,000]원 / 해외겸용 [20,000]원
📉 최소실적: 400000원

[1] 카테고리: 기본/전가맹점 > 선택형 (선택형)
   📝 AI 요약: 삼성 iD SELECT ALL 카드에서 선택한 옵션에 따라 다양한 할인 혜택을 제공합니다.
   🏢 주요 가맹점: 국내 가맹점, 온라인쇼핑몰, 의료, 배달앱, 음식점, 편의점, 할인점, 주유
   ✅ 상세 조건: 전월 실적 조건 요약 정보 없음
   💡 혜택 상세: 국내 가맹점 0.7% 할인, 아파트 관리비 10% 할인, 통신 10% 할인, 교육 10% 할인, 온라인쇼핑몰 7% 할인, 의료 7% 할인, 배달앱 7% 할인, 음식점 7% 할인, 편의점 7% 할인, 할인점 7% 할인, 주유 7% 할인
   ⚠️ 실적 제외: ['전월 실적 계산에서 제외되는 항목 정보 없음']
-----------------------------------------------------------------------------
[2] 카테고리: 기본/전가맹점 > 할인 (할인)
   📝 AI 요약: 삼성 iD SELECT ALL 카드로 다양한 가맹점에서 할인 혜택을 제공합니다.
   🏢 주요 가맹점: 국내 가맹점, 아파트 관리비, 통신, 교육, 온라인쇼핑몰, 의료, 배달앱, 음식점, 편의점, 할인점, 주유
   ✅ 상세 조건: 전월 실적에 관계없이 할인 제공, 전월 실적 40만원 미만 시에도 혜택 제공
   💡 혜택 상세: {'국내 가맹점': '0.7% 할인', '아파트 관리비, 통신': '10% 할인', '교육': '10% 할인', '온라인쇼핑, 의료, 배달앱': '7% 할인', '음식점, 편의점, 할인점, 주유': '7% 할인'}
----------------------------